In [24]:
import pandas as pd
import numpy as np

In [25]:
def f(rank):
    return 1/np.sqrt(rank)

In [31]:
def ranking_calculation(df, f, calc_sets=False):
    
    n_players = len(df.Joueur.unique())
    
    ### Résultat de chaque match
    results = []
    for match in df.Match.unique():
        scores = list(df[df["Match"]==match]["Scores"])
        
        result = -1
        if scores[0] > scores[1]:
            result = 1
        elif scores[0] == scores[1]:
            result = 0
            
        results.append(result)
        results.append(-result)

    df["Résultat"] = results

    ### Points de chaque match
    points = []
    for match in df.Match.unique():
        rank = list(df[df["Match"]==match]["Classement"])        
        sets = list(df[df["Match"]==match]["Scores"])
        result = list(df[df["Match"]==match]["Résultat"])
        for i in range(2):
            if calc_sets==False:
                point = max(3*np.sqrt(n_players)*result[i]*f(rank[1-i]), 0)
                points.append(point) 
            else:
                point = sets[i]*f(rank[1-i]) + (result[i]+1)
                points.append(point)
                
        
    df["Points"] = points
    
    ### Calcul des stats de chaque joueur
    df_ranks = pd.DataFrame(columns={"Rang", "Joueur"})
    
    df_ranks["Joueur"] = df.Joueur.unique()
    played, wins, draws, defeats, rate, points = [], [], [], [], [], []
    for player in df_ranks["Joueur"]:
        nb_played = df[df["Joueur"]==player].shape[0]
        played.append(nb_played)
        nb_wins = list(df[df["Joueur"]==player]["Résultat"]).count(1)
        wins.append(nb_wins)
        nb_draws = list(df[df["Joueur"]==player]["Résultat"]).count(0)
        draws.append(nb_draws)
        nb_defeats = list(df[df["Joueur"]==player]["Résultat"]).count(-1)
        defeats.append(nb_defeats)
        rate.append("{}%".format(int(nb_wins/nb_played*100)))
        points.append(round(sum(list(df[df["Joueur"]==player]["Points"])), 2))

    df_ranks["Joués"] = played
    df_ranks["Victoires"] = wins
    df_ranks["Nuls"] = draws
    df_ranks["Défaites"] = defeats
    df_ranks["Taux de succès"] = rate
    df_ranks["Points"] = points

    df_ranks.sort_values(by=["Points", "Taux de succès"], ascending=[False, False], inplace=True)
    
    df_ranks["Rang"] = range(1, n_players+1)
    
    return df_ranks

In [32]:
def ranking_squash(filename, f, calc_sets):
    
    df_matchs = pd.read_csv(filename, sep=';', decimal=',')
    
    if df_matchs.Classement.isna().any():
                
        df_previous_matchs = df_matchs.dropna().copy()

        df_previous_rank = ranking_calculation(df_previous_matchs, f, calc_sets)
        
        
        # Ajout des nouveaux joueurs
        count_new_players = len(df_matchs.Joueur.unique()) - len(df_previous_rank.Joueur.unique())
        
        if count_new_players != 0:
            print("There are {} new players".format(count_new_players))
            for player in df_matchs.Joueur.unique():
                if player not in df_previous_rank.Joueur.unique():
                    new_player = [player, len(df_previous_rank.Joueur.unique())+1] + [0]*6
                    row = pd.Series(new_player, index=df_previous_rank.columns)
                    df_previous_rank = df_previous_rank.append(row, ignore_index=True)

                    
        # Remplissage avec les nouveaux classements
        for i in range(df_matchs.shape[0]):
            if np.isnan(df_matchs["Classement"][i]):
                player = df_matchs["Joueur"][i]
                previous_rank = int(df_previous_rank[df_previous_rank["Joueur"]==player]["Rang"])
                df_matchs["Classement"][i] = previous_rank

        # Export de l'historique mis à jour
        df_matchs.to_csv(filename, sep=';', decimal=',', columns=["Match", "Joueur", "Classement", "Scores"], index=False)
    
    # Classement final
    df_rank = ranking_calculation(df_matchs, f, calc_sets)
    
    return df_rank

In [33]:
rank = ranking_squash("Historique.csv", f, True)
rank

Joueur  Rang  Joués  Victoires  Nuls  Défaites Taux de succès  Points
3      KRa     1     11         10     0         1            90%   32.75
1    Colas     2      4          4     0         0           100%   13.62
2       LL     3      5          2     0         3            40%    7.71
5  Gessien     4      4          2     0         2            50%    7.34
0    Marin     5      7          1     0         6            14%    6.02
6   Mateos     6      6          1     0         5            16%    4.13
7      PLN     7      3          1     0         2            33%    3.50
4   Sutter     8      2          0     0         2             0%    0.82

In [34]:
df_points = pd.DataFrame(columns={"Joueur", "Points à gagner (sur un set)"})
df_points["Joueur"] = rank.Joueur
n_players = len(list(df_points["Joueur"]))
df_points["Points à gagner (sur un set)"] = [round(f(i),2) for i in range(1, n_players + 1)]
df_points["Points à gagner (sur un match)"] = df_points["Points à gagner (sur un set)"]*3 + 2

df_points

Joueur  Points à gagner (sur un set)  Points à gagner (sur un match)
3      KRa                          1.00                            5.00
1    Colas                          0.71                            4.13
2       LL                          0.58                            3.74
5  Gessien                          0.50                            3.50
0    Marin                          0.45                            3.35
6   Mateos                          0.41                            3.23
7      PLN                          0.38                            3.14
4   Sutter                          0.35                            3.05